In [ ]:
import time
import random
import numpy as np
import pandas as pd

## Pandas Performance

**Konsten, och vikten, av att utföra operationer på lämpligt sätt**

In [ ]:
demand_profile_df = pd.read_csv('../data/demand_profile.csv')

In [ ]:
demand_profile_df.head(10)

In [ ]:
demand_profile_df.info()

_____

## Lite uppvärmning

Låt oss omvandla datumn kolumnen till rätt datatype (datetime)

**Omvandla till datetime rakt på**

In [ ]:
%%timeit -r 5  # utför koden nedan 5 gånger, och ta medelvärdet av tiden det tar

pd.to_datetime(demand_profile_df['date_time'])


**Omvandla med på förhand angivet format**

In [ ]:
%%timeit -r 5

pd.to_datetime(demand_profile_df['date_time'], format='%d/%m/%y %H:%M')


In [ ]:
# Ok nice, omvandla nu kolumnen till datetime med den bättre av meoderna ovan

demand_profile_df['date_time'] = pd.to_datetime(demand_profile_df['date_time'], format='%d/%m/%y %H:%M')


______

Låt oss nu skapa ett större dataset.

In [ ]:
# Create a sample DataFrame


n = 1000000  # en miljon rader

df = pd.DataFrame({
                    'A': np.random.randint(0, 100, n),
                    'B': np.random.randint(0, 100, n),
                    'C' : np.random.randint(0, 100, n)
                  })

df


_____

Anta nu att vi vill utföra en funktion, rad för rad i våran dataframe.

Exempelvis att vi vill summera kolumnerna A,B,C - rad för rad.

**Bad Practice #1: for-loop**

Det absolut **sämsta** sättet att lösa vårt problem på - EJ REKOMMENDERAT!

In [ ]:
result = []


start = time.time()

for i in range(len(df)):
    
    result.append(df.loc[i, 'A'] + df.loc[i, 'B'] + df.loc[i, 'C'])

end = time.time()

df['Sum'] = result

print(f"For loop execution time: {end - start:.4f} seconds")

**Bad Practice #2: df.iterrows()**

Pandas har en inbyggt metod (iterrows) för att loopa igenom rader i en dataframe. Detta är dock fortfarande **mycket** långsamt, och bör undvikas.



In [ ]:
result = []

start = time.time()
for index, row in df.iterrows():               # loops through the index and rows of the dataframe

    result.append(row['A']+row['B']+row['C'])
end = time.time()

df['Sum'] = result

print(f"iterrows execution time: {end - start:.4f} seconds")

**Better Practice #1: df.apply()**

Pandas har en inbyggd metod (apply) för att tillämpa en funktion på en dataframe. Detta är snabbare och effektivare metod, och bör användas istället för for-loops och iterrows.

In [ ]:
def compute_sum(row):
    return row['A'] + row['B'] + row['C']

start = time.time()

df.apply(compute_sum, axis=1)   # axis=1 means apply the function to each row

end = time.time()

print(f"apply execution time: {end - start:.4f} seconds")

**Best Practice #1: Vectorized Operations**

In [ ]:
start = time.time()

df['Sum'] = df['A'] + df['B'] + df['C']

end = time.time()

print(f"Vectorized execution time: {end - start:.4f} seconds")

_____

Anta att vi nu vill göra något lite mer komplicerat än att bara summera kolumnerna.

Låt oss återgå till vår ursprungliga dataframe.


In [ ]:
demand_profile_df


Anta att vi vill ll skapa en ny kolumn, som anger den faktiska kostnaden för den förbrukade elen.

Som ni kanska känner till kostar el olika mycket per kwh beroende på tid på dygnet.

Anta vidare att fölande gäller:

* **Peak-tid** är mellan kl 17 och 24, och priset är då 28 öre / kWh.
* **Off-peak-tid** är mellan kl 00 och 07, och priset är då 18 öre / kWh.
* **Shoulder-tid** är mellan kl 07 och 17, och priset är då 22 öre / kWh.

Vi vill nu skapa en ny kolumn *cost*, som anger respektive totalpris priset för den förbrukade elen i varje rad data ovan.

Option 1: med apply.

Funkar, men är inte bäst.

In [ ]:
def calculate_cost(row):

    if row['date_time'].hour >= 17 and row['date_time'].hour <= 24:
         return row['energy_kwh'] * 0.28
    
    elif row['date_time'].hour >= 0 and row['date_time'].hour <= 7:
         return row['energy_kwh'] * 0.18
    
    else:
         return row['energy_kwh'] * 0.22
    

demand_profile_df['cost'] = demand_profile_df.apply(calculate_cost, axis=1)

demand_profile_df


Man skulle kunna for-loopa, men det su*er! Apply skulle varit bättre, men låt oss istället försöka med det bästa - **vektoriserat!**  

In [ ]:
peak_demand_filter = demand_profile_df['date_time'].dt.hour.between(17, 24)
off_peak_demand_filter = demand_profile_df['date_time'].dt.hour.between(0, 7)
shoulder_demand_filter = ~(peak_demand_filter | off_peak_demand_filter)

demand_profile_df.loc[peak_demand_filter, 'cost'] = demand_profile_df.loc[peak_demand_filter, 'energy_kwh'] * 0.28
demand_profile_df.loc[off_peak_demand_filter, 'cost'] = demand_profile_df.loc[off_peak_demand_filter, 'energy_kwh'] * 0.18
demand_profile_df.loc[shoulder_demand_filter, 'cost'] = demand_profile_df.loc[shoulder_demand_filter, 'energy_kwh'] * 0.22

_____

Du kan läsa mer på följande site!

https://realpython.com/fast-flexible-pandas/